In [1]:
import sys
sys.path.append('../')
from read_data_functions import *
from plot_functions import *
from calculations import *
plt.style.use('../Style.mplstyle')
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # suppress warnings

In [9]:
path = '../../../Data/2025/250929-251003_Ulva/'
gcms_data = import_data(f'{path}GC-MS/', '', None, None, None)
gcms_data = gcms_data['250929-251003_Ulva_results']
flow_data = import_data(f'{path}Absorbent tube flows/', '', 'Time', '%d-%m-%Y %H:%M:%S', 0)

print(gcms_data.keys())

Index(['Name', 'Data File', 'Type', 'Level', 'Acq. Date-Time',
       'Ethene, 1,1-dichloro--copy Results', 'Methylene chloride Results',
       'Ethene, 1,1-dichloro- Results', 'Ethane, 1,1-dichloro- Results',
       'Trichloromethane Results', 'Ethane, 1,1,1-trichloro- Results',
       'Carbon Tetrachloride Results', 'Ethane, 1,2-dichloro- Results',
       'Trichloroethylene Results', 'Propane, 1,2-dichloro- Results',
       'Methane, bromodichloro- Results', 'Ethene, (2-chloroethoxy)- Results',
       '1-Propene, 1,3-dichloro-, (E)- Results',
       '1-Propene, 1,3-dichloro-, (E)--copy Results',
       'Ethane, 1,1,2-trichloro- Results', 'Tetrachloroethylene Results',
       'Methane, dibromochloro- Results', 'Benzene, chloro- Results',
       'Methane, tribromo- Results', 'Ethane, 1,1,2,2-tetrachloro- Results',
       'Methane, diiodo- Results', 'Benzene, 1,2-dichloro--copy Results',
       'Benzene, 1,2-dichloro--copy2 Results',
       'Benzene, 1,2-dichloro- Results'],
      dtyp

In [ ]:
timestamps = [['2025-09-29 17:11', '2025-09-29 19:11'],   # [P] = 3.9 uM, R1
              
              ['2025-09-30 09:37', '2025-09-30 11:34'],   # Blank1
              ['2025-09-30 12:41', '2025-09-30 14:38'],   # [P] = 0.02 uM, R2
              ['2025-09-30 16:15', '2025-09-30 18:14'],   # [P] = 19 uM, R1 

              ['2025-10-01 09:33', '2025-10-01 11:31'],   # [P] = 3.9 uM, R2
              ['2025-10-01 12:44', '2025-10-01 14:46'],   # Blank2
              ['2025-10-01 15:49', '2025-10-01 17:49'],   # [P] = 0.7 uM, R2

              ['2025-10-02 09:53', '2025-10-02 12:06'],   # [P] = 19 uM, R2
              ['2025-10-02 13:24', '2025-10-02 15:27'],   # [P] = 3.9 uM, R3
              ['2025-10-02 16:52', '2025-10-02 18:57'],   # [P] = 0.02 uM, R3

              ['2025-10-03 10:01', '2025-10-03 12:03'],   # [P] = 0.7 uM, R3  
              ['2025-10-03 13:18', '2025-10-03 15:17'],   # [P] = 19 uM, R3
              ['2025-10-03 16:59', '2025-10-03 18:59']    # Blank3
]

flow_keys = ['250929_P39_Ulva_A028727T_A028689C', 
             '250930_Blank1_Ulva_A029641T_A028596C', '250930_P002_Ulva_A028572T_A028052C', '250930_P19_Ulva_A028476T_A030827C', 
             '251001_P39_Ulva_A029668T_A030947C', '251001_Blank2_Ulva_A030818T_A031826C', '251001_P07_Ulva_A028453T_A029151C',
             '251002_P19_Ulva_A029717T_A029836C', '251002_P39_Ulva_A028586T_A029651C', '251002_P002_Ulva_A029658T_A030854C',
             '251003_P07_Ulva_A029817T_A029691C', '251003_P19_Ulva_A029803T_A028597C', '251003_Blank3_Ulva_A029154T_A028440C']

def GCMS_concentration_calculation(flow_data, flow_dict_keys, flow_key, timestamps, gcms_data, sorbent_type, V_inj):
    mean_flow = []
    exp_duration = []
    for time, key in zip(timestamps, flow_dict_keys):
        time_filtered_flow = time_filtered_conc(flow_data[key], [flow_key], time)
        mean_flow.append(np.array(time_filtered_flow[flow_key]).mean())
        elapsed = pd.to_datetime(time[1]) - pd.to_datetime(time[0])
        exp_duration.append(elapsed.total_seconds() / 60)
    
    gcms_concentrations = pd.DataFrame()

    if sorbent_type == 'TA' or sorbent_type == 'CA':
        for index, row in gcms_data.iterrows():
            concentration = (V_inj*10**(-6) * row[gcms_data.keys()[5:]]*10**(-3)) / (mean_flow[index]*10**(-3) * exp_duration[index])
            gcms_concentrations[row['Name']] = concentration

    if sorbent_type == 'TA and CA':
        for index, row in gcms_data.iterrows():
            for j, key in enumerate(flow_dict_keys):
                if key.split('_')[0] in row['Name'] and key.split('_')[1] in row['Name']:
                    concentration = (V_inj*10**(-6) * row[gcms_data.keys()[5:]]*10**(3)) / (mean_flow[j]*10**(-3) * exp_duration[j]) # unit of ug/m^3
                    gcms_concentrations[row['Name']] = concentration
    
    gcms_concentrations = gcms_concentrations.T
    gcms_concentrations.columns = ['Experiment'] + gcms_data.keys()[5:]

    return gcms_concentrations

gcms = GCMS_concentration_calculation(flow_data, flow_keys, ' Flow [L/min]', timestamps, gcms_data, 'TA and CA', 0.5)

